In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
np.random.seed(777)
import math
import keras
import keras.backend as K
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout, concatenate
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score
from keras.regularizers import l2
from keras.applications.xception import Xception, preprocess_input
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications import DenseNet201
from keras.applications import DenseNet121

from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import SeparableConv2D, AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["axes.grid"] = False
plt.rcParams.update({'font.size': 20})

Using TensorFlow backend.


In [3]:
train_dir = 'data/bioimaging/train/'
test_dir = "data/bioimaging/test/"

extracted_features_dir = "extracted_features/"
model_name = "bioimaging_test20_concate"

In [4]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0
dim_ordering: tf


In [5]:
batch_size = 32
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 1000

In [6]:
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

print("*"*30)
for root,dirs,files in os.walk(test_dir):
    print (root, len(files))

data/bioimaging/train/ 0
data/bioimaging/train/carcinoma 5696
data/bioimaging/train/non-carcinoma 5696
******************************
data/bioimaging/test/ 0
data/bioimaging/test/carcinoma 36
data/bioimaging/test/non-carcinoma 35


In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.25,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 8544 images belonging to 2 classes.
Found 2848 images belonging to 2 classes.
Found 71 images belonging to 2 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

nb_test_samples = len(test_generator.filenames)
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

# nb_test_without_aug_samples = len(test_generator_without_aug.filenames)
# predict_size_test_without_aug = int(math.ceil(nb_test_without_aug_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
# print("nb_test_without_aug_samples:", nb_test_without_aug_samples)
# print("predict_size_test_without_aug_samples:", predict_size_test_without_aug)
print("nb_test_samples:", nb_test_samples)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 8544
nb_validation_samples: 2848

predict_size_train: 267
predict_size_validation: 89
nb_test_samples: 71
predict_size_test: 3

 num_classes: 2


In [9]:
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))


reset_keras_tf_session()

In [10]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [11]:
print("Training Data Shape : {0}".format(train_data.shape))
print("Training Data label Shape : {0}".format(train_labels.shape))

print("Test Data Shape : {0}".format(test_data.shape))
print("Test Data label Shape : {0}".format(test_labels.shape))

Training Data Shape : (8544, 2592)
Training Data label Shape : (8544, 2)
Test Data Shape : (71, 2592)
Test Data label Shape : (71, 2)


### KFold Cross Validation

In [28]:
from sklearn.metrics import accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score, classification_report

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import cross_val_score


k_fold = KFold(n_splits=10, shuffle=True, random_state=5)

### Decision Tree Classifier

In [13]:
clf = DecisionTreeClassifier()
clf.fit(train_data, train_labels)
print("Train accuracy", clf.score(train_data, train_labels))
print("Validation accuracy", clf.score(validation_data, validation_labels ))
print("Test accuracy", clf.score(test_data, test_labels))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(test_labels, y_test_pred)
print('DecisionTree Classifier test accuracies %.4f' % (clf_test))

print(classification_report(test_labels, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(test_labels, y_test_pred))

Train accuracy 1.0
Validation accuracy 0.925561797752809
Test accuracy 0.7183098591549296
DecisionTree Classifier test accuracies 0.7183
              precision    recall  f1-score   support

           0       0.74      0.69      0.71        36
           1       0.70      0.74      0.72        35

   micro avg       0.72      0.72      0.72        71
   macro avg       0.72      0.72      0.72        71
weighted avg       0.72      0.72      0.72        71
 samples avg       0.72      0.72      0.72        71

0.7183098591549296


In [14]:
clf = DecisionTreeClassifier()
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, test_labels, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 64.6429 and (STDEV 0.1485)
Best result for fold 2
Best accuracy is 0.8571428571428571
Scores of all folds: [0.75       0.42857143 0.85714286 0.85714286 0.57142857 0.57142857
 0.57142857 0.71428571 0.71428571 0.42857143]
Accuracy: 0.65 (+/- 0.30)


### Random Forest Classifier

In [15]:
clf = RandomForestClassifier(n_estimators=5)
clf.fit(train_data, train_labels)
print("Train accuracy", clf.score(train_data, train_labels))
print("Validation accuracy", clf.score(validation_data, validation_labels ))
print("Test accuracy", clf.score(test_data, test_labels))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(test_labels, y_test_pred)
print('RandomForest Classifier test accuracies %.4f' % (clf_test))

print(classification_report(test_labels, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(test_labels, y_test_pred))

Train accuracy 0.9977762172284644
Validation accuracy 0.9610252808988764
Test accuracy 0.6901408450704225
RandomForest Classifier test accuracies 0.6901
              precision    recall  f1-score   support

           0       0.69      0.69      0.69        36
           1       0.69      0.69      0.69        35

   micro avg       0.69      0.69      0.69        71
   macro avg       0.69      0.69      0.69        71
weighted avg       0.69      0.69      0.69        71
 samples avg       0.69      0.69      0.69        71

0.6901408450704225


In [16]:
clf = RandomForestClassifier(n_estimators=5)
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, test_labels, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 56.2500 and (STDEV 0.1363)
Best result for fold 1
Best accuracy is 0.7142857142857143
Scores of all folds: [0.625      0.71428571 0.57142857 0.71428571 0.42857143 0.28571429
 0.57142857 0.71428571 0.57142857 0.42857143]
Accuracy: 0.56 (+/- 0.27)


___

In [17]:
y_test = np.asarray(test_labels)
y_test = np.argmax(y_test, axis=1)

In [18]:
y_validation = np.asarray(validation_labels)
y_validation = np.argmax(y_validation, axis=1)

In [19]:
y_train = np.asarray(train_labels)
y_train = np.argmax(y_train, axis=1)

### SVM

In [29]:
clf = SVC()
clf.fit(train_data, y_train)
print("Train accuracy", clf.score(train_data, y_train))
print("Validation accuracy", clf.score(validation_data, y_validation ))
print("Test accuracy", clf.score(test_data, y_test))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(y_test, y_test_pred)
print('SVM Classifier test accuracies %.4f' % (clf_test))

print(classification_report(y_test, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(y_test, y_test_pred))

Train accuracy 0.877691947565543
Validation accuracy 0.8672752808988764
Test accuracy 0.7323943661971831
SVM Classifier test accuracies 0.7324
              precision    recall  f1-score   support

           0       0.73      0.75      0.74        36
           1       0.74      0.71      0.72        35

   micro avg       0.73      0.73      0.73        71
   macro avg       0.73      0.73      0.73        71
weighted avg       0.73      0.73      0.73        71

0.7323943661971831


In [30]:
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, y_test, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 39.4643 and (STDEV 0.1243)
Best result for fold 7
Best accuracy is 0.7142857142857143
Scores of all folds: [0.375      0.42857143 0.42857143 0.28571429 0.28571429 0.42857143
 0.42857143 0.71428571 0.28571429 0.28571429]
Accuracy: 0.39 (+/- 0.25)


### XGBoost Classifier

In [21]:
clf = xgb.XGBClassifier()
clf.fit(train_data, y_train)
print("Train accuracy", clf.score(train_data, y_train))
print("Validation accuracy", clf.score(validation_data, y_validation ))
print("Test accuracy", clf.score(test_data, y_test))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(y_test, y_test_pred)
print('XGB Classifier test accuracies %.4f' % (clf_test))

print(classification_report(y_test, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(y_test, y_test_pred))

Train accuracy 0.9959035580524345
Validation accuracy 0.9841994382022472
Test accuracy 0.7887323943661971
XGB Classifier test accuracies 0.7887
              precision    recall  f1-score   support

           0       0.74      0.89      0.81        36
           1       0.86      0.69      0.76        35

   micro avg       0.79      0.79      0.79        71
   macro avg       0.80      0.79      0.79        71
weighted avg       0.80      0.79      0.79        71

0.7887323943661971


In [22]:
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, y_test, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 71.6071 and (STDEV 0.1299)
Best result for fold 0
Best accuracy is 0.875
Scores of all folds: [0.875      0.71428571 0.71428571 0.71428571 0.42857143 0.85714286
 0.85714286 0.71428571 0.71428571 0.57142857]
Accuracy: 0.72 (+/- 0.26)


### AdaBoost Classifier

In [23]:
clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators = 300 )
clf.fit(train_data, y_train)
print("Train accuracy", clf.score(train_data, y_train))
print("Validation accuracy", clf.score(validation_data, y_validation ))
print("Test accuracy", clf.score(test_data, y_test))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(y_test, y_test_pred)
print('AdaBoost Classifier test accuracies %.4f' % (clf_test))

print(classification_report(y_test, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(y_test, y_test_pred))

Train accuracy 1.0
Validation accuracy 0.9290730337078652
Test accuracy 0.6338028169014085
AdaBoost Classifier test accuracies 0.6338
              precision    recall  f1-score   support

           0       0.62      0.72      0.67        36
           1       0.66      0.54      0.59        35

   micro avg       0.63      0.63      0.63        71
   macro avg       0.64      0.63      0.63        71
weighted avg       0.64      0.63      0.63        71

0.6338028169014085


In [24]:
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, y_test, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 65.0000 and (STDEV 0.1643)
Best result for fold 1
Best accuracy is 0.8571428571428571
Scores of all folds: [0.5        0.85714286 0.85714286 0.71428571 0.57142857 0.71428571
 0.57142857 0.71428571 0.71428571 0.28571429]
Accuracy: 0.65 (+/- 0.33)


### Bagging Classifier

In [25]:
clf = BaggingClassifier(RandomForestClassifier(), n_estimators = 300 )
clf.fit(train_data, y_train)
print("Train accuracy", clf.score(train_data, y_train))
print("Validation accuracy", clf.score(validation_data, y_validation ))
print("Test accuracy", clf.score(test_data, y_test))

y_test_pred = clf.predict(test_data)
clf_test = accuracy_score(y_test, y_test_pred)
print('AdaBoost Classifier test accuracies %.4f' % (clf_test))

print(classification_report(y_test, y_test_pred))

# print(confusion_matrix(test_labels, y_test_pred))
print(accuracy_score(y_test, y_test_pred))

Train accuracy 1.0
Validation accuracy 0.9887640449438202
Test accuracy 0.8169014084507042
AdaBoost Classifier test accuracies 0.8169
              precision    recall  f1-score   support

           0       0.77      0.92      0.84        36
           1       0.89      0.71      0.79        35

   micro avg       0.82      0.82      0.82        71
   macro avg       0.83      0.82      0.81        71
weighted avg       0.83      0.82      0.81        71

0.8169014084507042


In [26]:
scoring = 'accuracy'
scores = cross_val_score(clf, test_data, y_test, cv=k_fold, n_jobs=1, scoring=scoring)

print ("Scores Mean: %.4f and (STDEV %.4f)" % (np.mean(scores)*100, np.std(scores)))
print ("Best result for fold %s" % np.argmax(scores))
print ("Best accuracy is", (scores[np.argmax(scores)]))
print ("Scores of all folds:", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores Mean: 71.9643 and (STDEV 0.1524)
Best result for fold 2
Best accuracy is 1.0
Scores of all folds: [0.625      0.71428571 1.         0.71428571 0.42857143 0.71428571
 0.85714286 0.85714286 0.71428571 0.57142857]
Accuracy: 0.72 (+/- 0.30)
